In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [4]:
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
X = df.v2
Y = df.v1
le = LabelEncoder()
# trans à des chiffres
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [8]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [10]:
# max_words = 1000
# max_len = 150
# tok = Tokenizer(num_words=max_words)
# tok.fit_on_texts(X_train)
# sequences = tok.texts_to_sequences(X_train)
# sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [11]:
sequences_matrix

array([[  0,   0,   0, ...,  35,  23, 132],
       [  0,   0,   0, ...,  38,  55, 136],
       [  0,   0,   0, ...,  14,   8, 358],
       ...,
       [  0,   0,   0, ...,   9, 191, 145],
       [  0,   0,   0, ..., 724,  15,   4],
       [  0,   0,   0, ...,   0,   0,  48]], dtype=int32)

In [12]:
X_train.shape

(4457,)

In [13]:
sequences_matrix.shape

(4457, 150)

In [14]:
Y_train.shape

(4457, 1)

In [15]:
sequences_matrix

array([[  0,   0,   0, ...,  35,  23, 132],
       [  0,   0,   0, ...,  38,  55, 136],
       [  0,   0,   0, ...,  14,   8, 358],
       ...,
       [  0,   0,   0, ...,   9, 191, 145],
       [  0,   0,   0, ..., 724,  15,   4],
       [  0,   0,   0, ...,   0,   0,  48]], dtype=int32)

In [16]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [17]:
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [49]:
# model.fit(sequences_matrix,Y_train,batch_size=128,epochs=3,
#           validation_split=0.2,callbacks=[EarlyStopping(monitor='',min_delta=0.0001)])

In [50]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the checkpoint callback
checkpoint = ModelCheckpoint('model_checkpoint.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model with checkpoint callback
model.fit(sequences_matrix, Y_train, batch_size=128, epochs=3, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', min_delta=0.0001), checkpoint])

Epoch 1/3
28/28 [==============================] - 10s 345ms/step - loss: 0.0549 - accuracy: 0.9913 - val_loss: 0.0598 - val_accuracy: 0.9854
Epoch 2/3
28/28 [==============================] - 7s 244ms/step - loss: 0.0470 - accuracy: 0.9930 - val_loss: 0.0860 - val_accuracy: 0.9877


In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [52]:
score = model.evaluate(test_sequences_matrix,Y_test)

35/35 [==============================] - 1s 23ms/step - loss: 0.1399 - accuracy: 0.9865


In [53]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(score[0],score[1]))

Test set
  Loss: 0.140
  Accuracy: 0.987


In [54]:
from keras.models import save_model
save_model(model,'model.h5')


In [56]:
from keras.models import load_model
# Load the model from the file
model1 = load_model('model.h5')
print(model1.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   